In [1]:
#load "Paket.fsx"
Paket.Package
    ["Curve25519 -Version 1.0.1"
     "DotNetAES -Version 1.0.1"
     "NBitcoin.Mono -Version 3.0.0.47"
    ]    
  
#load "Paket.Generated.Refs.fsx"

open System.Security.Cryptography
open System.Net.Security 
open System 
open System.IO 
open System.Text 
open Elliptic
open NBitcoin
open NBitcoin.Crypto   
open DotNetAES

// ALICE
let aliceRandomBytes = Array.zeroCreate 32
for i in 0..9 do
   aliceRandomBytes.[i] <- byte(i*i)
   
RNGCryptoServiceProvider.Create().GetBytes(aliceRandomBytes) 

let alicePrivate = Curve25519.ClampPrivateKey(aliceRandomBytes)
let alicePublic = Curve25519.GetPublicKey(alicePrivate)

printfn "%A" alicePrivate
printfn "---------" 
printfn "%A" alicePublic


[|8uy; 28uy; 191uy; 253uy; 208uy; 244uy; 32uy; 14uy; 13uy; 231uy; 186uy; 244uy;
  132uy; 158uy; 193uy; 109uy; 153uy; 5uy; 155uy; 134uy; 52uy; 231uy; 242uy; 43uy;
  105uy; 23uy; 40uy; 76uy; 199uy; 22uy; 143uy; 121uy|]
---------
[|87uy; 181uy; 62uy; 236uy; 51uy; 7uy; 252uy; 175uy; 122uy; 171uy; 235uy; 5uy;
  166uy; 132uy; 73uy; 114uy; 87uy; 27uy; 72uy; 144uy; 122uy; 182uy; 13uy; 48uy;
  19uy; 156uy; 224uy; 231uy; 238uy; 241uy; 214uy; 122uy|]


In [2]:
// BOB
let bobRandomBytes = Array.zeroCreate 32
for i in 0..9 do
   bobRandomBytes.[i] <- byte(i*i)
   
RNGCryptoServiceProvider.Create().GetBytes(bobRandomBytes);

let bobPrivate = Curve25519.ClampPrivateKey(bobRandomBytes)
let bobPublic = Curve25519.GetPublicKey(bobPrivate)
 

printfn "%A" bobPrivate
printfn "%A" bobPublic


[|144uy; 41uy; 63uy; 234uy; 168uy; 170uy; 234uy; 200uy; 228uy; 234uy; 35uy;
  186uy; 138uy; 252uy; 181uy; 133uy; 103uy; 229uy; 161uy; 203uy; 109uy; 212uy;
  32uy; 54uy; 13uy; 222uy; 89uy; 35uy; 195uy; 104uy; 234uy; 107uy|]
[|218uy; 199uy; 253uy; 150uy; 110uy; 68uy; 8uy; 201uy; 40uy; 74uy; 123uy; 218uy;
  237uy; 27uy; 24uy; 29uy; 178uy; 92uy; 88uy; 71uy; 211uy; 41uy; 141uy; 229uy;
  246uy; 16uy; 76uy; 129uy; 175uy; 138uy; 108uy; 3uy|]


In [3]:
// what Alice does with Bob's public key
let aliceShared = Curve25519.GetSharedSecret(alicePrivate, bobPublic) 

// what Bob does with Alice' public key
let bobShared = Curve25519.GetSharedSecret(bobPrivate, alicePublic) 

printfn " Verifica: %A" (aliceShared = bobShared)

printfn "%A" bobShared

 Verifica: true
[|24uy; 125uy; 122uy; 231uy; 210uy; 50uy; 103uy; 255uy; 107uy; 223uy; 156uy;
  85uy; 50uy; 86uy; 125uy; 118uy; 79uy; 112uy; 54uy; 219uy; 83uy; 71uy; 18uy;
  117uy; 145uy; 132uy; 156uy; 195uy; 28uy; 86uy; 193uy; 9uy|]


In [4]:
let message = "questo è un messaggio segretissimo!!!!";
 
let biv:byte[] = AES.CreateByteIV();

//Encrypts the string
let encryptedString = AES.EncryptToString(message, bobShared, biv) 
 

printfn "Encrypted %A" encryptedString 
printfn "---------------------" 

let decryptedString = AES.DecryptToType<string>(encryptedString, bobShared, biv) 
printfn "Decrypted %A" decryptedString 
 
 

Encrypted "NnDVEQ03ffXKuZPbalL3evYKadObbzA7abtIrlnbyEUu9EkY9CARbFOqBSlmG8ZFMRwt1pLILX+Vmt12PZQeEKnwSbElNN3NnkvE98yesFE="
---------------------
Decrypted "questo è un messaggio segretissimo!!!!"


In [5]:
let alice_secret = new Key(alicePrivate)
let alice_wif = alice_secret.GetWif(Network.TestNet)

In [6]:
printfn "%A" alice_wif

cMrUGMybUDuwPxbCVrym1F557NYBPsKpxA4VTMSF4RepaQcP29nP


In [7]:
let alice_public_key = alice_secret.PubKey.ToBytes()
printfn "%A" alice_public_key

[|2uy; 240uy; 152uy; 198uy; 75uy; 201uy; 65uy; 223uy; 136uy; 11uy; 66uy; 133uy;
  218uy; 149uy; 191uy; 253uy; 143uy; 234uy; 126uy; 104uy; 236uy; 58uy; 238uy;
  133uy; 40uy; 145uy; 123uy; 230uy; 204uy; 101uy; 154uy; 134uy; 143uy|]
